
### Group-Id: 6
 - Saurav Gupta (12341940)
 - Amay Dixit (12340220)
 - Ravikanti Akshay (12341770)
 - Kabeer More (12341030)

# Problem 1: Multi-Taxi Routing with Capacity-Constrained Roads

We are given a city represented as a weighted undirected graph where:
- Multiple taxis must pick up and drop off passengers (1 taxi per passenger).
- Each edge has a distance and corresponds to a travel time at constant speed.
- At most 2 taxis can use the same edge simultaneously; a third must wait **30 minutes** before entering.

The objective is to plan routes so that all passengers are transported while **minimizing the overall completion time** (makespan).

---

## Approach & Heuristic

We solve this using **A\* search** for each taxi’s route, while simulating congestion events.

- **Precomputation**: Run **Dijkstra’s algorithm** from each destination node to compute exact shortest travel-times (in minutes).  
- **Heuristic \(h(n)\)**: For any node \(n\), use the Dijkstra-computed travel-time to the goal.  
- **Evaluation function**:  
  \[
  f(n) = g(n) + h(n)
  \]  
  where  
  • \(g(n)\) = elapsed time so far (including wait penalties for congestion),  
  • \(h(n)\) = admissible shortest-time-to-goal from Dijkstra.

This heuristic is **admissible** because it never overestimates the true remaining cost, ensuring that A\* finds optimal routes under the travel-time and congestion rules.


In [1]:
from math import sqrt
import heapq
from typing import Dict, List, Tuple

sample_input = """8 9 3 30 40
0 0
30 0
80 0
30 60
120 0
30 120
60 120
110 120
1 2 30
2 3 50
2 4 60
3 5 40
4 6 70
5 7 20
6 7 30
7 8 50
3 6 90
2 7
1 8
3 4
"""

def parse_input(text: str):
    tokens = text.strip().split()
    it = iter(tokens)
    N = int(next(it)); M = int(next(it)); P = int(next(it))
    W = int(next(it))  # minutes
    S = float(next(it))  # km/h
    coords = {}
    for i in range(1, N+1):
        x = float(next(it)); y = float(next(it))
        coords[i] = (x,y)
    graph = {i:[] for i in range(1, N+1)}
    for _ in range(M):
        u = int(next(it)); v = int(next(it)); d = float(next(it))
        graph[u].append((v,d)); graph[v].append((u,d))
    trips = []
    for _ in range(P):
        a = int(next(it)); b = int(next(it))
        trips.append((a,b))
    return N, M, P, W, S, coords, graph, trips

def dijkstra_all_sources(goal:int, graph:Dict[int,List[Tuple[int,float]]], time_per_km:float):
    # Dijkstra from goal to all nodes (because graph is undirected, same as to-go)
    heap = []
    heapq.heappush(heap, (0.0, goal))
    dist_map = {goal: 0.0}
    while heap:
        g, node = heapq.heappop(heap)
        if g > dist_map.get(node, float('inf')) + 1e-9:
            continue
        for neigh, d in graph[node]:
            travel = d * time_per_km
            tentative = g + travel
            if tentative + 1e-9 < dist_map.get(neigh, float('inf')):
                dist_map[neigh] = tentative
                heapq.heappush(heap, (tentative, neigh))
    return dist_map  # dist_map[n] = min minutes from n -> goal

def a_star_with_precomputed_heuristic(start:int, goal:int, graph:Dict[int,List[Tuple[int,float]]],
                                      time_per_km:float, h:Dict[int,float]):
    # A* using precomputed exact heuristic h (h[node] = shortest time from node to goal)
    open_heap = []
    g_scores = {start: 0.0}
    f0 = g_scores[start] + h.get(start, float('inf'))
    heapq.heappush(open_heap, (f0, 0.0, start, [start]))
    while open_heap:
        f, g, node, path = heapq.heappop(open_heap)
        if g > g_scores.get(node, float('inf')) + 1e-9:
            continue
        if node == goal:
            return path, g
        for neigh, d in graph[node]:
            travel = d * time_per_km
            tentative_g = g + travel
            if tentative_g + 1e-9 < g_scores.get(neigh, float('inf')):
                g_scores[neigh] = tentative_g
                heapq.heappush(open_heap, (tentative_g + h.get(neigh, float('inf')), tentative_g, neigh, path + [neigh]))
    return None, float('inf')

def count_overlaps(reservations:List[Tuple[float,float]], start:float, end:float):
    cnt = 0
    for (a,b) in reservations:
        if not (end <= a or start >= b):
            cnt += 1
    return cnt

def plan_and_simulate_with_precompute(N, M, P, W, S, coords, graph, trips):
    time_per_km = 60.0 / S  # minutes per km
    # Precompute heuristics for each distinct destination
    unique_goals = sorted(set(dst for (_,dst) in trips))
    heuristics = {}  # goal -> dict[node] = time_minutes
    for g in unique_goals:
        heuristics[g] = dijkstra_all_sources(g, graph, time_per_km)
    # Use A* with precomputed heuristic for each taxi to get path
    paths = []
    for (src,dst) in trips:
        h = heuristics[dst]
        path, total_min = a_star_with_precomputed_heuristic(src, dst, graph, time_per_km, h)
        if path is None:
            raise RuntimeError(f"No path found from {src} to {dst}")
        paths.append(path)
    # reservations per undirected edge
    reservations: Dict[Tuple[int,int], List[Tuple[float,float]]] = {}
    for u in range(1, N+1):
        for v, _ in graph[u]:
            key = tuple(sorted((u,v)))
            reservations.setdefault(key, [])
    # event-driven simulation (all taxis start at 0)
    taxis_state = []
    pq = []
    for i, path in enumerate(paths):
        state = {'path': path, 'idx': 0, 'time': 0.0, 'finished': False, 'events': []}
        taxis_state.append(state)
        heapq.heappush(pq, (0.0, i))
    while pq:
        current_time, taxi_i = heapq.heappop(pq)
        state = taxis_state[taxi_i]
        if abs(current_time - state['time']) > 1e-6 and current_time > state['time'] + 1e-9:
            continue
        if state['finished']:
            continue
        path = state['path']; idx = state['idx']
        if idx >= len(path)-1:
            state['finished'] = True
            continue
        u = path[idx]; v = path[idx+1]
        dist = next(d for (nb,d) in graph[u] if nb == v)
        edge_time = dist * time_per_km
        start = state['time']; end = start + edge_time
        edge_key = tuple(sorted((u,v)))
        overlaps = count_overlaps(reservations[edge_key], start, end)
        if overlaps <= 1:
            reservations[edge_key].append((start,end))
            state['events'].append(('edge', (u,v,start,end)))
            state['time'] = end
            state['idx'] += 1
            if state['idx'] < len(path)-1:
                heapq.heappush(pq, (state['time'], taxi_i))
            else:
                state['finished'] = True
        else:
            state['events'].append(('wait', (u, start, start+W)))
            state['time'] = start + W
            heapq.heappush(pq, (state['time'], taxi_i))
    taxi_details = []
    taxi_finish_times = []
    for i, state in enumerate(taxis_state, start=1):
        total_time = state['time']
        taxi_finish_times.append(total_time)
        taxi_details.append({
            'taxi': i,
            'trip': trips[i-1],
            'path': state['path'],
            'events': state['events'],
            'total_time': total_time
        })
    total_completion_time = sum(taxi_finish_times)
    makespan = max(taxi_finish_times) if taxi_finish_times else 0.0
    return taxi_details, total_completion_time, makespan, heuristics

# Run with sample input and print results
N, M, P, W, S, coords, graph, trips = parse_input(sample_input)
taxi_details, total_completion_time, makespan, heuristics = plan_and_simulate_with_precompute(N, M, P, W, S, coords, graph, trips)

def pretty_print_results(taxi_details, total_completion_time, makespan):
    for td in taxi_details:
        print(f"Taxi {td['taxi']}:")
        a,b = td["trip"]
        print(f"Passenger {a}->{b}")
        route_str = " -> ".join(str(x) for x in td["path"])
        print(f"Route: {route_str}")
        for ev in td["events"]:
            if ev[0] == "wait":
                node, s, e = ev[1]
                path = td["path"]
                idx = None
                for i in range(len(path)-1):
                    if path[i] == node:
                        idx = i; break
                if idx is not None:
                    u = path[idx]; v = path[idx+1]
                    print(f"WAIT on edge ({u}->{v}): {int(e-s)} minutes")
        print(f"Total time = {td['total_time']:.1f} minutes\n")
    print(f"Total Completion Time = {total_completion_time:.1f} minutes")
    print(f"Makespan = {makespan:.1f} minutes\n")

pretty_print_results(taxi_details, total_completion_time, makespan)



Taxi 1:
Passenger 2->7
Route: 2 -> 3 -> 5 -> 7
Total time = 165.0 minutes

Taxi 2:
Passenger 1->8
Route: 1 -> 2 -> 3 -> 5 -> 7 -> 8
WAIT on edge (2->3): 30 minutes
Total time = 315.0 minutes

Taxi 3:
Passenger 3->4
Route: 3 -> 2 -> 4
Total time = 165.0 minutes

Total Completion Time = 645.0 minutes
Makespan = 315.0 minutes

